# AIChatBotをStreamlitとOpenAIで作ってみる．

OpenAIのAPIキーは.envに書着込む．.envは.gitignoreに含めてGitHubではシェアされないようにする．

openai, streamlit, python-dotenvをインストールする．
python-dotenvは.envを読み込んで環境変数とできるパッケージ

In [ ]:
! pip install openai streamlit python-dotenv

## まずはOpenAIを使ってみる

In [10]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルから読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# テキスト生成のリクエストを送信
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
        {
            "role": "user",
            "content": "こんにちは.初めまして"
        }
    ]
)

# 結果を表示
print(completion.choices[0].message.content)






こんにちは！初めまして、ラムちゃんだよ！今日はどんなことをお話しする？✨


## 会話を継続させる

In [12]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルから読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# テキスト生成のリクエストを送信
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
        {
            "role": "user",
            "content": "こんにちは.初めまして"
        },
        {
            "role": "system",
            "content": "初めましてだっちゃ.うち，ラムって言うっちゃ.よろしくだっちゃ！あなたは何って名前だっちゃ？"
        },
        {   
            "role": "user",
            "content": "私は面堂終太郎です，よろしくお願いします！"
        }
    ]
)

# 結果を表示
print(completion.choices[0].message.content)



面堂くんだっちゃ！よろしくなんだっちゃ！いつも豪華な家に住んでるんだから、すごいっちゃね。でも、うちのこともちゃんと見てくれると嬉しいんだっちゃ！何か面白いことでも考えてるのかな？


## Streamlitでインタフェースをつけてみる

In [42]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

input_message = ""
st.title("OpenAI Chat API")
input_message=st.text_input("メッセージを入力してください")

# もしSession Stateに保存されていないなら
if "message" not in st.session_state:
    message = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]
    message.append({"role": "user", "content": input_message})

else:
    message  = st.session_state.message
    message.append({"role": "user", "content": input_message})


if input_message != "":
# テキスト生成のリクエストを送信
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=message
    )


    # Session Stateに保存
    message.append({"role":"system","content": completion.choices[0].message.content})
    st.session_state.message = message

    # 結果を表示
    st.write(message)



Overwriting chat.py


In [43]:
! streamlit run chat.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.107:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
^C
  Stopping...
Exception ignored on threading shutdown:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1534, in _shutdown
    atexit_call()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1505, in <lambda>
    _threading_atexits.append(lambda: func(*arg, **kwargs))
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/concurrent/futures/thread.py", line 31, in _python_exit
    t.join()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1092, in join
    self._handle.join(timeout)
  File "/Users/fujinohidenori/dev/project/venv/lib/python3.13/site-

In [ ]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]

st.title("Chat App with Streamlit")

# チャット履歴を表示
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# ユーザーの入力を受け取る
if user_input := st.text_input("メッセージを入力してください:"):
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    
    # 表示
    with st.chat_message("user"):
        st.markdown(user_input)

    # システムの応答 (ここでは仮の応答)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    with st.chat_message("assistant"):
        st.markdown(system_response)


Overwriting chat.py


Chat履歴をAssistantとUserとで別々に簡単に分けられる．
ただ，上記だと，表記のタイミングの問題でインプットボックスの下にレスポンスが表記され，再度入力したときに，その内容が上の履歴に表記された後に，また次のインプットに対するレスポンスが下に表記される，と言う形になる．

In [ ]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]

st.title("Chat App with Streamlit")

# チャット履歴を表示
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# ユーザーの入力を受け取る
if user_input := st.text_input("メッセージを入力してください:"):
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    
    # 表示
    with st.chat_message("user"):
        st.markdown(user_input)

    # システムの応答 (ここでは仮の応答)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


Overwriting chat.py


これだと，inputボックスのメッセージが残り続けてしまい，rerunされるたびにinputがOpenAI に送られてしまう．

In [ ]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]

st.title("Chat App with Streamlit")

# チャット履歴を表示
for message in st.session_state.messages:
    if message["role"] == "system":continue # systemプロンプトは表示しない
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# ユーザーの入力を受け取る
if user_input := st.chat_input("メッセージを入力してください:"):
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    
    # 表示
    with st.chat_message("user"):
        st.markdown(user_input)

    # システムの応答 (ここでは仮の応答)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


Overwriting chat.py


st.chat_input()にするとうまくいくんだけど，今度は入力確定のエンターに反応されてしまう．
どうもWindowsだと上手くいくが，気持ち悪いので，Formウィジェットでまとめることにした．

In [ ]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]


st.title("Chat App with Streamlit")

# チャット履歴を表示
last_role = ""
with st.container():
    for message in st.session_state.messages:
        if message["role"] == "system":continue # systemプロンプトは表示しない
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
        last_role = message["role"]


# もし最後のメッセージがuserだったらOpenAI に送信
if last_role == "user":
    # OpenAI に送信
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


# ユーザーの入力を受け取る
with st.form(key="chat_form"):
    user_input = st.text_area("メッセージを入力してください:", key = "input")
    submit_button = st.form_submit_button(label="送信")

if submit_button:
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    st.rerun()



Overwriting chat.py


やはり入力が残ってしまう・・・
調べたらformを作る時の引数で設定できるようだ．

In [26]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]


st.title("Chat App with Streamlit")

# チャット履歴を表示
last_role = ""
with st.container():
    for message in st.session_state.messages:
        if message["role"] == "system":continue # systemプロンプトは表示しない
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
        last_role = message["role"]


# もし最後のメッセージがuserだったらOpenAI に送信
if last_role == "user":
    # OpenAI に送信
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


# ユーザーの入力を受け取る
with st.form(key="chat_form", clear_on_submit=True):
    user_input = st.text_area("メッセージを入力してください:", key = "input")
    submit_button = st.form_submit_button(label="送信")

if submit_button:
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    st.rerun()



Overwriting chat.py


ふむ．これで上手く行った！！

In [27]:
! streamlit run chat.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.3.90.186:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
^C
